# Core (Always run)

In [ ]:
import shap
import mygene
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score, learning_curve, cross_validate, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_recall_curve, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, label_binarize
from itertools import cycle
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, RFE, SelectFromModel
from sklearn.pipeline import Pipeline

Global Variables

In [ ]:
path_to_data = "data/"
dataset_file_name = "dataset.pq"

# Preprocess

## Load datasets

In [ ]:
df = pd.read_parquet(path_to_data + dataset_file_name)

## Cleaning

Transpose

In [ ]:
df = df.transpose()

print(f'Dataframe shape after transpose: {df.shape}')

df.head()

Apply subtypes

In [ ]:
excell_sheet_df = pd.read_excel('./assets/subtype_sheet.xlsx', sheet_name='RNA-Seq 1148')

for sample_id in df.index:
    print(f'Processing sample ID: {sample_id}')

    if sample_id in excell_sheet_df['Sample ID'].values:
        subtype = excell_sheet_df.loc[excell_sheet_df['Sample ID'] == sample_id, 'PAM50'].values[0]
        print(f'Subtype found: {subtype}')
        df.at[sample_id, 'Subtype'] = subtype

df.head()

Look for NaN

In [ ]:
if df.isna().sum().sum() > 0:
    print("Dataframe contains missing values. Dropping missing values.")
    print(f'Number of missing values: {df.isna().sum().sum()}')

    df = df.dropna()

    print("Missing values dropped.")
    print(f'Number of remaining missing values: {df.isna().sum().sum()}')
else:
    print("Dataframe does not contain missing values.")

# Exploratory Data Analysis (EDA)

Plot 1: Subtype distribution plot

More info about the subtypes in this paper: https://pmc.ncbi.nlm.nih.gov/articles/PMC6985186/

In [ ]:
plot_df = df.copy()

plt.figure(figsize=(10, 6))
sns.countplot(data=plot_df, x='Subtype', order=plot_df['Subtype'].value_counts().index)
plt.title('Distribution of Subtypes')
plt.xlabel('Subtype')
plt.ylabel('Count')
plt.show()

Plot 2: Scatter plot

Observation: Contains a few outliers, not entirely sure what to do about them.

https://stats.stackexchange.com/questions/533503/when-should-you-remove-outliers-entire-dataset-or-train-dataset

In [ ]:
x_log_transformed = np.log1p(plot_df.select_dtypes(include=np.number))

scaler = StandardScaler()
df_scaled = scaler.fit_transform(x_log_transformed)

PCA_model = PCA(n_components=2)
pca_result = PCA_model.fit_transform(df_scaled)
plot_df['PCA1'] = pca_result[:, 0]
plot_df['PCA2'] = pca_result[:, 1]
plt.figure(figsize=(10, 6))
sns.scatterplot(data=plot_df, x='PCA1', y='PCA2', hue='Subtype', palette='Set2')
plt.title('PCA Scatter Plot Colored by Subtype')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.legend(title='Subtype')
plt.show()

Plot 2.1: Scatter plot with outliers removed

In [ ]:
# Filter out outliers based on PCA1 and PCA2
filtered_plot_df = plot_df[plot_df['PCA1'] < 2000]

plt.figure(figsize=(10, 6))
sns.scatterplot(data=filtered_plot_df, x='PCA1', y='PCA2', hue='Subtype', palette='Set2')
plt.title('PCA Scatter Plot with PCA1 < 2000 Colored by Subtype')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.legend(title='Subtype')
plt.show()  

# Training

## Setup

Stratified K fold

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
n_jobs = -1

lr_base_model = LogisticRegression(max_iter=7500, random_state=42)
rf_base_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=n_jobs)
xgb_base_model = XGBClassifier(objective='multi:softmax', random_state=42, n_jobs=n_jobs, eval_metric='mlogloss')

Model training function

In [ ]:
def setup_pipeline(model, type: str) -> Pipeline:
    """Returns pipeline based on model and type of feature selection. Feature selection types: rfe, sbm, skb"""

    base_list = [
        ('scaler', StandardScaler()),
        ('variance_threshold', VarianceThreshold(threshold=0.0))
    ]

    if type == 'rfe':
        return Pipeline(base_list + [
            ('feature_selection', RFE(estimator=LogisticRegression(max_iter=1500, random_state=42), n_features_to_select=50, step=0.1)),
            ('model', model)
        ])
    elif type == 'sfm':
        return Pipeline(base_list + [
            ('feature_selection', SelectFromModel(estimator=RandomForestClassifier(n_estimators=100, random_state=42), max_features=50)),
            ('model', model)
        ])
    elif type == 'skb':
        return Pipeline(base_list + [
            ('feature_selection', SelectKBest(score_func=f_classif, k=50)),
            ('model', model)
        ])
    else:
        raise ValueError("Invalid feature selection type. Choose from 'lr', 'rf', or 'skb'.")
    
def print_score(scores):
    print(f"Scores for each fold: {scores}")
    print(f"Average score: {np.mean(scores)}")
    print(f"Standard deviation: {np.std(scores)}")

Labeling

In [ ]:
encoder = LabelEncoder()

y = encoder.fit_transform(df['Subtype'])
X = df.drop(columns=['Subtype'])

Normalization - log2

In [ ]:
X = np.log2(X + 1)

Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Training Logistic Regression using LR, RF and KBest for feature selection

Logistic Regression - RFE(LogisticRegression) Feature Selection

In [ ]:
logreg_pipeline_rfe = setup_pipeline(model=lr_base_model, type='rfe')

logreg_rfe_result = cross_validate(
    estimator=logreg_pipeline_rfe, 
    X=X_train, 
    y=y_train, 
    cv=skf, 
    scoring='f1_macro',
    n_jobs=n_jobs,
    return_train_score=True
)

print_score(logreg_rfe_result['test_score'])

Logistic Regression - SelectBestModel(RandomForest) Feature Selection

In [ ]:
logreg_pipeline_sfm = setup_pipeline(model=lr_base_model, type='sfm')

logreg_sfm_result = cross_validate(
    estimator=logreg_pipeline_sfm,
    X=X_train,
    y=y_train,
    cv=skf,
    scoring='f1_macro',
    n_jobs=n_jobs,
    return_train_score=True
)

print_score(logreg_sfm_result['test_score'])

Logistic Regression - SelectKBest Feature Selection

In [ ]:
logreg_pipeline_kbest = setup_pipeline(model=lr_base_model, type='skb')

logreg_kbest_result = cross_validate(
    estimator=logreg_pipeline_kbest,
    X=X_train,
    y=y_train,
    cv=skf,
    scoring='f1_macro',
    n_jobs=n_jobs,
    return_train_score=True
)

print_score(logreg_kbest_result['test_score'])

## Training Random Forest using LR, RF and KBest for feature selection

Random Forest - RFE(LogisticRegression) Feature Selection

In [ ]:
rf_pipeline_rfe = setup_pipeline(model=rf_base_model, type='rfe')

rf_rfe_result = cross_validate(
    estimator=rf_pipeline_rfe,
    X=X_train,
    y=y_train,
    cv=skf,
    scoring='f1_macro',
    n_jobs=n_jobs,
    return_train_score=True
)

print_score(rf_rfe_result['test_score'])

Random Forest - SelectBestModel(RandomForest) Feature Selection

In [ ]:
rf_pipeline_sfm = setup_pipeline(model=rf_base_model, type='sfm')

rf_sfm_result = cross_validate(
    estimator=rf_pipeline_sfm,
    X=X_train,
    y=y_train,
    cv=skf,
    scoring='f1_macro',
    n_jobs=n_jobs,
    return_train_score=True
)

print_score(rf_sfm_result['test_score'])

Random Forest - SelectKBest Feature Selection

In [ ]:
rf_pipeline_kbest = setup_pipeline(model=rf_base_model, type='skb')

rf_kbest_result = cross_validate(
    estimator=rf_pipeline_kbest,
    X=X_train,
    y=y_train,
    cv=skf,
    scoring='f1_macro',
    n_jobs=n_jobs,
    return_train_score=True
)

print_score(rf_kbest_result['test_score'])

## Training XGBoost using LR, RF and KBest for feature selection

XGBoost - RFE(LogisticRegression) Feature Selection

In [ ]:
xgb_pipeline_rfe = setup_pipeline(model=xgb_base_model, type='rfe')

xgb_rfe_result = cross_validate(
    xgb_pipeline_rfe,
    X_train,
    y_train,
    cv=skf,
    scoring='f1_macro',
    n_jobs=n_jobs,
    return_train_score=True
)

print_score(xgb_rfe_result['test_score'])

XGBoost - SelectBestModel(RandomForest) Feature Selection

In [ ]:
xgb_pipeline_sfm = setup_pipeline(model=xgb_base_model, type='sfm')

xgb_sfm_result = cross_validate(
    estimator=xgb_pipeline_sfm,
    X=X_train,
    y=y_train,
    cv=skf,
    scoring='f1_macro',
    n_jobs=n_jobs,
    return_train_score=True
)

print_score(xgb_sfm_result['test_score'])

XGBoost - SelectKBest Feature Selection

In [ ]:
xgb_pipeline_kbest = setup_pipeline(model=xgb_base_model, type='skb')

xgb_kbest_result = cross_validate(
    estimator=xgb_pipeline_kbest,
    X=X_train,
    y=y_train,
    cv=skf,
    scoring='f1_macro',
    n_jobs=n_jobs,
    return_train_score=True
)

print_score(xgb_kbest_result['test_score'])

## Final training score

In [ ]:
all_scores = {
    'logistic_regression_rfe': {
        'result': logreg_rfe_result,
        'base_model': lr_base_model,
        'type': 'rfe'
    },
    'logistic_regression_sfm': {
        'result': logreg_sfm_result,
        'base_model': lr_base_model,
        'type': 'sfm'
    },
    'logistic_regression_skb': {
        'result': logreg_kbest_result,
        'base_model': lr_base_model,
        'type': 'skb'
    },
    'random_forest_rfe': {
        'result': rf_rfe_result,
        'base_model': rf_base_model,
        'type': 'rfe'
    },
    'random_forest_sfm': {
        'result': rf_sfm_result,
        'base_model': rf_base_model,
        'type': 'sfm'
    },
    'random_forest_skb': {
        'result': rf_kbest_result,
        'base_model': rf_base_model,
        'type': 'skb'
    },
    'xgboost_rfe': {
        'result': xgb_rfe_result,
        'base_model': xgb_base_model,
        'type': 'rfe'
    },
    'xgboost_sfm': {
        'result': xgb_sfm_result,
        'base_model': xgb_base_model,
        'type': 'sfm'
    },
    'xgboost_skb': {
        'result': xgb_kbest_result,
        'base_model': xgb_base_model,
        'type': 'skb'
    }
}

results = []
for model_name, data in all_scores.items():
    val_scores = data['result']['test_score']
    train_scores = data['result']['train_score']

    mean_val_score = np.mean(val_scores)
    std_val_score = np.std(val_scores)
    mean_train_score = np.mean(train_scores)
    overfitting_gap = mean_train_score - mean_val_score

    results.append({
        'model': model_name,
        'mean_val_f1': mean_val_score,
        'std_dev': std_val_score,
        'mean_train_f1': mean_train_score,
        'overfitting_gap': overfitting_gap
    })

report_df = pd.DataFrame(results)

report_df = report_df.sort_values(by='overfitting_gap', ascending=True)

pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', None)

print(report_df.to_string(index=False))

# Training Results Analysis

Learning Curve plot

In [ ]:
best_model_name = report_df.iloc[0]['model']

best_pipeline = setup_pipeline(
    model=all_scores[best_model_name]['base_model'],
    type=all_scores[best_model_name]['type']
)

train_sizes, train_scores, test_scores = learning_curve(
    estimator=best_pipeline,
    X=X_train,
    y=y_train,
    cv=skf,
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 5)
)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.xlabel('Number of Training Samples')
plt.ylabel('F1-Macro Score')

plt.plot(train_sizes, train_scores_mean, 'o-', color='blue', label='Training score')
plt.plot(train_sizes, test_scores_mean, 'o-', color='green', label='Cross-validation score')

plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='blue')
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color='green')

plt.legend(loc='best')
plt.title(f'Learning Curve for Best Model: {best_model_name}')
plt.grid()
plt.show()

Confusion matrix

In [ ]:
cv_predict_scores = cross_val_predict(
    best_pipeline,
    X_train,
    y_train,
    cv=skf,
    n_jobs=n_jobs
)

cm = confusion_matrix(y_train, cv_predict_scores)
plt.figure(figsize=(10, 8))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.title(f'Confusion Matrix for Best Model: {best_model_name}')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

ROC Curve

In [ ]:
roc_curve_model = best_pipeline.fit(X_train, y_train)

y_train_proba = roc_curve_model.predict_proba(X_train)

y_train_binarized = label_binarize(y_train, classes=np.arange(len(encoder.classes_)))

plt.figure(figsize=(12, 8))
colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']

auc_scores = []

for i, (class_name, color) in enumerate(zip(encoder.classes_, colors)):
    fpr, tpr, _ = roc_curve(y_train_binarized[:, i], y_train_proba[:, i])
    roc_auc = auc(fpr, tpr)
    auc_scores.append(roc_auc)
    
    plt.plot(fpr, tpr, color=color, lw=2, 
             label=f'{class_name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier (AUC = 0.50)')

plt.xlabel('False Positive Rate', fontsize=13, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=13, fontweight='bold')
plt.title('ROC Curves for Multi-Class Classification\n(One-vs-Rest)', fontsize=15, fontweight='bold', pad=20)
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.tight_layout()
plt.show()

# Hyper Parameter Tuning

In [ ]:
n_features = [50, 75, 100]

param_grid = [
    {
        'model__penalty': ['l1'],
        'model__solver': ['saga'],
        'model__C': [0.01, 0.1, 1, 10, 50],
        'model__class_weight': ['balanced', None]
    },
    {
        'model__penalty': ['l2'],
        'model__solver': ['lbfgs', 'newton-cg', 'newton-cholesky'],
        'model__C': [0.01, 0.1, 1, 10, 50],
        'model__class_weight': ['balanced', None]
    }
]

# Insert N feature based on feature selection method
for grid in param_grid:
    if all_scores[best_model_name]['type'] == 'rfe':
        grid['feature_selection__n_features_to_select'] = n_features
    elif all_scores[best_model_name]['type'] == 'sfm':
        grid['feature_selection__max_features'] = n_features
    elif all_scores[best_model_name]['type'] == 'skb':
        grid['feature_selection__k'] = n_features
    else:
        raise ValueError("Invalid feature selection type. Choose from 'rfe', 'sfm', or 'skb'.")

grid_search = GridSearchCV(
    estimator=best_pipeline,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=skf,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print(f"Best F1-Macro Score: {grid_search.best_score_:.4f}")
print("Best Parameters Found:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_

# Hyper Parameter Tuning Results Analysis

Learning curve plot

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(
    estimator=best_model,
    X=X_train,
    y=y_train,
    cv=skf,
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 5)
)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.xlabel('Number of Training Samples')
plt.ylabel('F1-Macro Score')

plt.plot(train_sizes, train_scores_mean, 'o-', color='blue', label='Training score')
plt.plot(train_sizes, test_scores_mean, 'o-', color='green', label='Cross-validation score')

plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='blue')
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color='green')

plt.legend(loc='best')
plt.title(f'Learning Curve for Best Model: {best_model_name}')
plt.grid()
plt.show()

Confusion matrix

In [ ]:
cv_predict_scores = cross_val_predict(
    best_model,
    X_train,
    y_train,
    cv=skf,
    n_jobs=n_jobs
)

cm = confusion_matrix(y_train, cv_predict_scores)
plt.figure(figsize=(10, 8))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.title(f'Confusion Matrix for Best Model: {best_model_name}')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

ROC Curve

In [ ]:
roc_curve_model = best_pipeline.fit(X_train, y_train)

y_train_proba = roc_curve_model.predict_proba(X_train)

y_train_binarized = label_binarize(y_train, classes=np.arange(len(encoder.classes_)))

plt.figure(figsize=(12, 8))
colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']

auc_scores = []

for i, (class_name, color) in enumerate(zip(encoder.classes_, colors)):
    fpr, tpr, _ = roc_curve(y_train_binarized[:, i], y_train_proba[:, i])
    roc_auc = auc(fpr, tpr)
    auc_scores.append(roc_auc)
    
    plt.plot(fpr, tpr, color=color, lw=2, 
             label=f'{class_name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier (AUC = 0.50)')

plt.xlabel('False Positive Rate', fontsize=13, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=13, fontweight='bold')
plt.title('ROC Curves for Multi-Class Classification\n(One-vs-Rest)', fontsize=15, fontweight='bold', pad=20)
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.tight_layout()
plt.show()

# Shap Analysis

In [ ]:
scaler_step = best_model.named_steps['scaler']
vt_step = best_model.named_steps['variance_threshold']
selector_step = best_model.named_steps['feature_selection']
model_step = best_model.named_steps['model']

gene_ids_array = np.array(X_train.columns)

vt_mask = vt_step.get_support()
genes_after_vt = gene_ids_array[vt_mask]

skb_mask = selector_step.get_support()
final_selected_gene_ids = genes_after_vt[skb_mask]

final_selected_gene_ids_transformed = [gene_id.split('.')[0] for gene_id in final_selected_gene_ids]

mg = mygene.MyGeneInfo()
query_results = mg.querymany(
    final_selected_gene_ids_transformed,
    scopes='ensembl.gene', 
    fields='symbol', 
    species='human',
    verbose=False
)

gene_symbol_mapping = {res['query']: res.get('symbol', res['query']) for res in query_results}
final_feature_names = [gene_symbol_mapping.get(gene_id.split('.')[0]) for gene_id in final_selected_gene_ids]

X_train_scaled = scaler_step.transform(X_train)
X_train_vt = vt_step.transform(X_train_scaled)
X_train_transformed = selector_step.transform(X_train_vt)

In [ ]:
explainer = shap.LinearExplainer(model_step, X_train_transformed)
shap_values = explainer(X_train_transformed)

In [ ]:
global_importance_score = np.abs(shap_values.values).mean(axis=0).sum(axis=1)

feature_importance_series = pd.Series(global_importance_score, index=final_feature_names).sort_values(ascending=False)

print("Top 10 Important Features based on SHAP values:")
print(feature_importance_series.head(10))

Simple ROPN1 Influence by Subtype - Bar Chart

In [ ]:
ropn1_idx = final_feature_names.index('ROPN1')
class_names = encoder.classes_

influence_scores = []
for class_idx in range(len(class_names)):
    shap_vals = shap_values[:, class_idx, ropn1_idx].values
    influence_scores.append(np.mean(np.abs(shap_vals)))

plt.figure(figsize=(10, 6))
colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']
bars = plt.bar(class_names, influence_scores, color=colors, edgecolor='black', linewidth=2)

for bar, value in zip(bars, influence_scores):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.4f}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.ylabel('Mean Absolute SHAP Value\n(Influence Strength)', fontsize=13, fontweight='bold')
plt.xlabel('Subtype', fontsize=13, fontweight='bold')
plt.title('ROPN1 Gene: Influence by Cancer Subtype', fontsize=15, fontweight='bold', pad=20)
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()

print("ROPN1 Influence Ranking:")
print("-" * 40)
ranking = sorted(zip(class_names, influence_scores), key=lambda x: x[1], reverse=True)
for rank, (subtype, score) in enumerate(ranking, 1):
    print(f"{rank}. {subtype:10s}: {score:.4f}")

print(f"\n🎯 HIGHEST influence: {ranking[0][0]} ({ranking[0][1]:.4f})")

plt.show()